Displays unweighted plots from the corresponding Coffea outputs.  The plots are categorized according to the histogram name (dependant variable) and the tag category defined in the `TTbarResProcessor`.  
# NOTE: #
All QCD MC histograms are normalized directly to the data, as no corrections are applied via mistag analysis or modmass procedures anyways.  For a more realistic analysis, refer to `TTbarResCoffea_BkgEstAnalysis` notebook.

In [ ]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
from coffea import hist
from coffea import util
import numpy as np
import itertools

In [ ]:
dir1 = 'CoffeaOutputs/UnweightedOutputs/'
dir2 = 'CoffeaOutputsForCombine/Coffea_FirstRun/QCD/2016/noAPV/TTbarRes_0l_UL16postVFP_QCD.coffea'

QCD_unweighted = util.load('CoffeaOutputsForCombine/Coffea_FirstRun/QCD/2016/noAPV/TTbarRes_0l_UL16postVFP_QCD.coffea')
TTbar_unweighted = util.load('CoffeaOutputsForCombine/Coffea_FirstRun/TT/2016/noAPV/TTbarRes_0l_UL16postVFP_TTbar.coffea')
JetHT2016_unweighted = util.load('CoffeaOutputsForCombine/Coffea_FirstRun/JetHT/2016/TTbarRes_0l_JetHT2016_Data.coffea')
RSGluon1000_unweighted = util.load('CoffeaOutputsForCombine/Coffea_FirstRun/RSGluonToTT/2016/noAPV/TTbarRes_0l_UL16postVFP_RSGluon1000.coffea')
DM1000_unweighted = util.load('CoffeaOutputsForCombine/Coffea_FirstRun/ZprimeDMToTTbar/2016/noAPV/TTbarRes_0l_UL16postVFP_DM1000.coffea')

# The cutflow can be checked if desired 

In [ ]:
Datasets = {
    'QCD': QCD_unweighted,
    'TTbar': TTbar_unweighted,
    'JetHT2016': JetHT2016_unweighted,
    'RSGluon1000': RSGluon1000_unweighted,
    'DM1000': DM1000_unweighted
}

In [ ]:
for dataset,output in Datasets.items():
    print("-------" + dataset + " Cutflow--------")
    for i,j in output['cutflow'].items():        
        print( '%20s : %20s' % (i,j) )

## Make Save Directory

In [ ]:
def mkdir_p(mypath):
    '''Creates a directory. equivalent to using mkdir -p on the command line'''

    from errno import EEXIST
    from os import makedirs,path

    try:
        makedirs(mypath)
    except OSError as exc: # Python >2.5
        if exc.errno == EEXIST and path.isdir(mypath):
            pass
        else: raise

In [ ]:
def DoesDirectoryExist(mypath): #extra precaution (Probably overkill...)
    '''Checks to see if Directory exists before running mkdir_p'''
    import os.path
    from os import path
    
    if path.exists(mypath):
        pass
    else:
        mkdir_p(mypath)

# Prepare to Loop through Analysis Categories and Histograms

In [ ]:
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import warnings
import re # regular expressions
warnings.filterwarnings("ignore")

# ---- Reiterate categories ---- #
# ttagcats = ["AT&Pt", "at", "pret", "0t", "1t", ">=1t", "2t", ">=0t"] # 'AT&Pt' = anti-tagged j0 with t-tagged probe jet j1; 'pret' = t-tagged j0
ttagcats = ["AT&Pt", "at", "pret", ">=0t", "0t", ">=1t", "1t", "2t"]
btagcats = ["0b", "1b", "2b"]
ycats = ['cen', 'fwd']

list_of_cats = [ t+b+y for t,b,y in itertools.product( ttagcats, btagcats, ycats) ]

# ---- List the Histograms Here ---- #
list_of_hists_4vector = ('SDmass', 'tau32', 'jetmass', 'ttbarmass', 'jetpt', 'jeteta', 'jetphi', 'jety', 'jetdy')

In [ ]:
maindirectory = os.getcwd() # prepare to locally save images 

# Luminosity, Cross Sections and Scale Factors

In [ ]:
Nevts2016 = 583876623 #625516390. # from dasgoclient 
# Nevts2017 = 410461585.
# Nevts2018 = 676328827.
# Nevts = Nevts2016 + Nevts2017 + Nevts2018
Nevts_sf2016 = Nevts2016/JetHT2016_unweighted['cutflow']['all events']
print(Nevts_sf2016)

In [ ]:
""" ---------------- Luminosity and Cross Sections ---------------- """
Lum     = 137190. # pb^-1 from https://twiki.cern.ch/twiki/bin/viewauth/CMS/PdmVAnalysisSummaryTable
Lum2016 = 35920. / Nevts_sf2016

ttbar_BR = 0.45 #Calculated using PDG 2019
ttbar_xs = 831.76  #pb Monte Carlo already includes xs in event weight!!
# ttbar_sf2016 = ttbar_xs*Lum2016*ttbar_BR/TTbar_unweighted['cutflow']['sumw2'] 
ttbar_sf2016 = ttbar_xs*Lum2016/TTbar_unweighted['cutflow']['sumw'] 
print(ttbar_sf2016)

qcd_xs = 1370000000.0 #pb From https://cms-gen-dev.cern.ch/xsdb Set to one as a guess that these MC recieved the same treatment as ttbar
qcd_sf2016 = qcd_xs*Lum2016/QCD_unweighted['cutflow']['sumw']

# ---- https://cms-gen-dev.cern.ch/xsdb/?columns=67108863&currentPage=0&pageSize=0&searchQuery=process_name%3DRSGluon ---- #
# ---- May need to be double checked ---- #
RSGluon1000_xs = 20.95 # Ignore as a guess and simply divide by the total number of generated events
RSGluon1500_xs = 3.679
RSGluon2000_xs = 0.9429
RSGluon2500_xs = 0.3039
RSGluon3000_xs = 0.1163
RSGluon3500_xs = 0.05132
RSGluon4000_xs = 0.02545
RSGluon4500_xs = 0.01422
RSGluon5000_xs = 0.008634

DM1000_xs = 2.222

RSGluon1000_sf2016 = Lum2016/RSGluon1000_unweighted['cutflow']['all events'] # weights are 1 (maybe?)

DM1000_sf2016 = Lum2016/DM1000_unweighted['cutflow']['all events']

# Plotting Options

In [ ]:
stack_ttbar_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'red'}
line_rsg1000_opts = {'color': 'purple'}
line_dm1000_opts = {'color': 'black'}
stack_background_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'yellow'}
stack_error_opts = {'label':'Stat. Unc.', 'hatch':'///', 'facecolor':'None', 'edgecolor':(0,0,0,.5), 'linewidth': 0, 'label':'_nolegend_'}
data_err_opts = {'linestyle': 'none', 'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1}

# Data and Unweighted MC Plots
### NOTE that SDmass used axes called jetmass, so code will get confused unless exception is made for SDmass

In [ ]:
# for dataset,output in Datasets.items():
for ihist in list_of_hists_4vector:
    SaveDirectory = maindirectory + '/PreliminaryAnalysisPlots/' + ihist + '/NormalizedToLuminosity/' # split histograms into subdirectories
    DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
    for icat in list_of_cats: 
        plt.rcParams.update({
        'font.size': 14,
        'axes.titlesize': 18,
        'axes.labelsize': 18,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12
        })
        fig, (ax, rax) = plt.subplots(
            nrows=2,
            ncols=1,
            figsize=(7,7),
            gridspec_kw={"height_ratios": (3, 1)},
            sharex=True
        )
        fig.subplots_adjust(hspace=.07)
        title = ihist + '  ' + icat
        #filename = ihist + '_' + icat + '_LinearScale.' + 'png'
        filename = ihist + '_' + icat + '.' + 'png'
        legend_labels = {'labels':['QCD Bkg (Sim.)', r'$t\bar{t}$ Sim.', r'RSKK Gluon $1TeV$ Sim.', r'DM Med. $1TeV$ Sim.', 'Data', '_nolegend_', '_nolegend_', '_nolegend_'],
                         'loc': 'upper right',
                         'fontsize': 'xx-small'}
        
#         ttbar_sf2016 = ttbar_xs*Lum2016*ttbar_BR/TTbar_unweighted['cutflow'][icat]
#         qcd_sf2016 = qcd_xs*Lum2016/QCD_unweighted['cutflow'][icat]
#         RSGluon1000_sf2016 = RSGluon1000_xs*Lum2016/RSGluon1000_unweighted['cutflow'][icat]

        
#    ===================================================================================================================
#    DDDD       A    TTTTTTT    A        H     H IIIIIII   SSSSS TTTTTTT   OOO   GGGGGGG RRRRRR     A    M     M   SSSSS     
#    D   D     A A      T      A A       H     H    I     S         T     O   O  G       R     R   A A   MM   MM  S          
#    D    D   A   A     T     A   A      H     H    I    S          T    O     O G       R     R  A   A  M M M M S           
#    D     D  AAAAA     T     AAAAA      HHHHHHH    I     SSSSS     T    O     O G  GGGG RRRRRR   AAAAA  M  M  M  SSSSS      
#    D    D  A     A    T    A     A     H     H    I          S    T    O     O G     G R   R   A     A M     M       S     
#    D   D   A     A    T    A     A     H     H    I         S     T     O   O  G     G R    R  A     A M     M      S      
#    DDDD    A     A    T    A     A     H     H IIIIIII SSSSS      T      OOO    GGGGG  R     R A     A M     M SSSSS
#    ===================================================================================================================
        
        Data_hist = JetHT2016_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'JetHT2016_Data')
        Data_hist.scale(Nevts_sf2016) #scale according to number of events in dataset
        
#    ===================================================================================================
#    N     N   OOO   RRRRRR  M     M    A    L       IIIIIII ZZZZZZZ EEEEEEE       QQQ     CCCC  DDDD        
#    NN    N  O   O  R     R MM   MM   A A   L          I         Z  E            Q   Q   C      D   D       
#    N N   N O     O R     R M M M M  A   A  L          I        Z   E           Q     Q C       D    D      
#    N  N  N O     O RRRRRR  M  M  M  AAAAA  L          I       Z    EEEEEEE     Q     Q C       D     D     
#    N   N N O     O R   R   M     M A     A L          I      Z     E           Q   Q Q C       D    D      
#    N    NN  O   O  R    R  M     M A     A L          I     Z      E            Q   Q   C      D   D       
#    N     N   OOO   R     R M     M A     A LLLLLLL IIIIIII ZZZZZZZ EEEEEEE       QQQ Q   CCCC  DDDD
#    ===================================================================================================
        
        QCD_hist = QCD_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'UL16postVFP_QCD')
        QCD_hist.scale(qcd_sf2016) #scaled according to luminosity

#    ===================================================================================================================
#    N     N   OOO   RRRRRR  M     M    A    L       IIIIIII ZZZZZZZ EEEEEEE     TTTTTTT TTTTTTT BBBBBB     A    RRRRRR      
#    NN    N  O   O  R     R MM   MM   A A   L          I         Z  E              T       T    B     B   A A   R     R     
#    N N   N O     O R     R M M M M  A   A  L          I        Z   E              T       T    B     B  A   A  R     R     
#    N  N  N O     O RRRRRR  M  M  M  AAAAA  L          I       Z    EEEEEEE        T       T    BBBBBB   AAAAA  RRRRRR      
#    N   N N O     O R   R   M     M A     A L          I      Z     E              T       T    B     B A     A R   R       
#    N    NN  O   O  R    R  M     M A     A L          I     Z      E              T       T    B     B A     A R    R      
#    N     N   OOO   R     R M     M A     A LLLLLLL IIIIIII ZZZZZZZ EEEEEEE        T       T    BBBBBB  A     A R     R 
#    ===================================================================================================================
        
        TTbar_hist = TTbar_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'UL16postVFP_TTbar')
        TTbar_hist.scale(ttbar_sf2016) #scaled according to luminosity

#    ===================================================================================================================================
#    N     N   OOO   RRRRRR  M     M    A    L       IIIIIII ZZZZZZZ EEEEEEE     RRRRRR    SSSSS GGGGGGG L       U     U   OOO   N     N     
#    NN    N  O   O  R     R MM   MM   A A   L          I         Z  E           R     R  S      G       L       U     U  O   O  NN    N     
#    N N   N O     O R     R M M M M  A   A  L          I        Z   E           R     R S       G       L       U     U O     O N N   N     
#    N  N  N O     O RRRRRR  M  M  M  AAAAA  L          I       Z    EEEEEEE     RRRRRR   SSSSS  G  GGGG L       U     U O     O N  N  N     
#    N   N N O     O R   R   M     M A     A L          I      Z     E           R   R         S G     G L       U     U O     O N   N N     
#    N    NN  O   O  R    R  M     M A     A L          I     Z      E           R    R       S  G     G L        U   U   O   O  N    NN     
#    N     N   OOO   R     R M     M A     A LLLLLLL IIIIIII ZZZZZZZ EEEEEEE     R     R SSSSS    GGGGG  LLLLLLL   UUU     OOO   N     N 
#    ===================================================================================================================================

        RSG1000_hist = RSGluon1000_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'UL16postVFP_RSGluon1000')
        RSG1000_hist.scale(RSGluon1000_sf2016) #scaled according to luminosity  
        
#    ===========================================================================================
#    N     N   OOO   RRRRRR  M     M    A    L       IIIIIII ZZZZZZZ EEEEEEE     DDDD    M     M     
#    NN    N  O   O  R     R MM   MM   A A   L          I         Z  E           D   D   MM   MM     
#    N N   N O     O R     R M M M M  A   A  L          I        Z   E           D    D  M M M M     
#    N  N  N O     O RRRRRR  M  M  M  AAAAA  L          I       Z    EEEEEEE     D     D M  M  M     
#    N   N N O     O R   R   M     M A     A L          I      Z     E           D    D  M     M     
#    N    NN  O   O  R    R  M     M A     A L          I     Z      E           D   D   M     M     
#    N     N   OOO   R     R M     M A     A LLLLLLL IIIIIII ZZZZZZZ EEEEEEE     DDDD    M     M
#    ===========================================================================================
        
        DM1000_hist = DM1000_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'UL16postVFP_DM1000')
        DM1000_hist.scale(DM1000_sf2016) #scaled according to luminosity
            
#    ===========================================================================
#    M     M    A    K     K EEEEEEE     PPPPPP  L         OOO   TTTTTTT   SSSSS     
#    MM   MM   A A   K   K   E           P     P L        O   O     T     S          
#    M M M M  A   A  K K     E           P     P L       O     O    T    S           
#    M  M  M  AAAAA  KKk     EEEEEEE     PPPPPP  L       O     O    T     SSSSS      
#    M     M A     A K  K    E           P       L       O     O    T          S     
#    M     M A     A K   K   E           P       L        O   O     T         S      
#    M     M A     A K   K   EEEEEEE     P       LLLLLLL   OOO      T    SSSSS 
#    ===========================================================================

        MC_hist = TTbar_hist.copy()
        MC_hist.add(QCD_hist)
        
        # ---- Extract both the data and MC events and from histograms ---- #
        if ihist == 'SDmass':
            NtotalMC = np.sum(MC_hist.integrate('jetmass').values())
            NtotalData = np.sum(Data_hist.integrate('jetmass').values())
        else:
            NtotalMC = np.sum(MC_hist.integrate(ihist).values())
            NtotalData = np.sum(Data_hist.integrate(ihist).values())
        NtotalMC = [k for k in NtotalMC.values()]
        NtotalData = [l for l in NtotalData.values()]
        
        # ---- Normalize the total MC histogram directly to the data (for aesthetic purposes only!) ---- #
        # -------- Unweighted simulation of the background alone greatly overestimates ------- #
        if NtotalMC[0] > 0.:
            MC_hist.scale(NtotalData[0]/NtotalMC[0])
            # TTbar_hist.scale(NtotalData[0]/NtotalMC[0])
        else:
            MC_hist.scale(0.)
            # TTbar_hist.scale(0.)

        #---- Plot Data ----#
        #-----------------------------------------------------------------#
        hist.plot1d(Data_hist, ax=ax, clear=False, 
                    error_opts=data_err_opts)
        
        #---- Plot Total MC (simulated QCD + SM ttbar background)----#
        #-----------------------------------------------------------------#
        hist.plot1d(MC_hist, ax=ax, clear=False,
                    fill_opts=stack_background_opts, error_opts=stack_error_opts)
        
        #---- Plot TTbar MC for comparison ---- #
        #-----------------------------------------------------------------#    
        hist.plot1d(TTbar_hist, ax=ax, clear=False,
                    fill_opts=stack_ttbar_opts, error_opts=stack_error_opts)
        
        #---- Plot RSG MC for comparison ---- #
        #-----------------------------------------------------------------#   
        hist.plot1d(RSG1000_hist, ax=ax, clear=False,
                    line_opts=line_rsg1000_opts, error_opts=stack_error_opts, legend_opts=legend_labels)
        
        
        #---- Plot DMM MC for comparison ----#
        #-----------------------------------------------------------------#
        hist.plot1d(DM1000_hist, ax=ax, clear=False,
                    line_opts=line_dm1000_opts, error_opts=stack_error_opts, legend_opts=legend_labels)
        
        ax.set_ylim(bottom=0.1)
        ax.set_yscale('log')
        ax.set_ylabel('Events')
        ax.set_xlabel(None)
        ax.set_title(title)
        leg = ax.legend(labels=[r'QCD Sim. Bkg', r'$t\bar{t}$ Sim.', r'RSKK Gluon $1TeV$ Sim.', r'DM Med. $1TeV$ Sim.', r'Data'])
        
        
        #---- Plot Ratio ----#
        hist.plotratio(num = Data_hist, denom = MC_hist, ax = rax,
                       error_opts={'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1},
                       unc = 'num')
        rax.set_ylabel('Data/MC')
        rax.axhline(y=1, color='k', linestyle=':')
        rax.set_ylim(0,2)
        
        if ihist == 'ttbarmass':
            rax.set_xlim(700,5000)
        if ihist == 'jetpt':
            rax.set_xlim(400,3000)
        if ihist == 'jeteta':
            rax.set_xlim(-3,3)
        if ihist == 'tau32':
            rax.set_xlim(0,1.2)

        #---- Labeling ----#
        Lint = str(Lum2016*.001) # Integrated Luminosity
        lumi = plt.text(1.15, 1.07, Lint[:6] + " fb$^{-1}$",
                fontsize=16,
                horizontalalignment='right',
                verticalalignment='top',
                transform=ax.transAxes
               )
#         plt.savefig(SaveDirectory+filename, bbox_inches="tight")
        # print(SaveDirectory+filename)

In [ ]:
# # for dataset,output in Datasets.items():
# # SaveDirectory = maindirectory + '/PreliminaryAnalysisPlots/' + ihist + '/NormalizedToLuminosity/' # split histograms into subdirectories
# # DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already

# ihist = 'ttbarmass'
# for icat in list_of_cats: 
#     plt.rcParams.update({
#     'font.size': 14,
#     'axes.titlesize': 18,
#     'axes.labelsize': 18,
#     'xtick.labelsize': 12,
#     'ytick.labelsize': 12
#     })
#     fig, (ax, rax) = plt.subplots(
#         nrows=2,
#         ncols=1,
#         figsize=(7,7),
#         gridspec_kw={"height_ratios": (3, 1)},
#         sharex=True
#     )
#     fig.subplots_adjust(hspace=.07)
#     title = ihist + '  ' + icat
#     #filename = ihist + '_' + icat + '_LinearScale.' + 'png'
#     filename = ihist + '_' + icat + '.' + 'png'
#     legend_labels = {'labels':['QCD Bkg (Sim.)', r'$t\bar{t}$ Sim.', r'RSKK Gluon $1TeV$ Sim.', r'DM Med. $1TeV$ Sim.', 'Data', '_nolegend_', '_nolegend_', '_nolegend_'],
#                      'loc': 'upper right',
#                      'fontsize': 'xx-small'}

# #         ttbar_sf2016 = ttbar_xs*Lum2016*ttbar_BR/TTbar_unweighted['cutflow'][icat]
# #         qcd_sf2016 = qcd_xs*Lum2016/QCD_unweighted['cutflow'][icat]
# #         RSGluon1000_sf2016 = RSGluon1000_xs*Lum2016/RSGluon1000_unweighted['cutflow'][icat]


# #    ===================================================================================================================
# #    DDDD       A    TTTTTTT    A        H     H IIIIIII   SSSSS TTTTTTT   OOO   GGGGGGG RRRRRR     A    M     M   SSSSS     
# #    D   D     A A      T      A A       H     H    I     S         T     O   O  G       R     R   A A   MM   MM  S          
# #    D    D   A   A     T     A   A      H     H    I    S          T    O     O G       R     R  A   A  M M M M S           
# #    D     D  AAAAA     T     AAAAA      HHHHHHH    I     SSSSS     T    O     O G  GGGG RRRRRR   AAAAA  M  M  M  SSSSS      
# #    D    D  A     A    T    A     A     H     H    I          S    T    O     O G     G R   R   A     A M     M       S     
# #    D   D   A     A    T    A     A     H     H    I         S     T     O   O  G     G R    R  A     A M     M      S      
# #    DDDD    A     A    T    A     A     H     H IIIIIII SSSSS      T      OOO    GGGGG  R     R A     A M     M SSSSS
# #    ===================================================================================================================

#     Data_hist = JetHT2016_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'JetHT2016_Data')
#     Data_hist.scale(Nevts_sf2016) #scale according to number of events in dataset

# #    ===================================================================================================
# #    N     N   OOO   RRRRRR  M     M    A    L       IIIIIII ZZZZZZZ EEEEEEE       QQQ     CCCC  DDDD        
# #    NN    N  O   O  R     R MM   MM   A A   L          I         Z  E            Q   Q   C      D   D       
# #    N N   N O     O R     R M M M M  A   A  L          I        Z   E           Q     Q C       D    D      
# #    N  N  N O     O RRRRRR  M  M  M  AAAAA  L          I       Z    EEEEEEE     Q     Q C       D     D     
# #    N   N N O     O R   R   M     M A     A L          I      Z     E           Q   Q Q C       D    D      
# #    N    NN  O   O  R    R  M     M A     A L          I     Z      E            Q   Q   C      D   D       
# #    N     N   OOO   R     R M     M A     A LLLLLLL IIIIIII ZZZZZZZ EEEEEEE       QQQ Q   CCCC  DDDD
# #    ===================================================================================================

#     QCD_hist = QCD_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'UL16postVFP_QCD')
#     QCD_hist.scale(qcd_sf2016) #scaled according to luminosity

# #    ===================================================================================================================
# #    N     N   OOO   RRRRRR  M     M    A    L       IIIIIII ZZZZZZZ EEEEEEE     TTTTTTT TTTTTTT BBBBBB     A    RRRRRR      
# #    NN    N  O   O  R     R MM   MM   A A   L          I         Z  E              T       T    B     B   A A   R     R     
# #    N N   N O     O R     R M M M M  A   A  L          I        Z   E              T       T    B     B  A   A  R     R     
# #    N  N  N O     O RRRRRR  M  M  M  AAAAA  L          I       Z    EEEEEEE        T       T    BBBBBB   AAAAA  RRRRRR      
# #    N   N N O     O R   R   M     M A     A L          I      Z     E              T       T    B     B A     A R   R       
# #    N    NN  O   O  R    R  M     M A     A L          I     Z      E              T       T    B     B A     A R    R      
# #    N     N   OOO   R     R M     M A     A LLLLLLL IIIIIII ZZZZZZZ EEEEEEE        T       T    BBBBBB  A     A R     R 
# #    ===================================================================================================================

#     TTbar_hist = TTbar_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'UL16postVFP_TTbar')
#     TTbar_hist.scale(ttbar_sf2016) #scaled according to luminosity

# #    ===================================================================================================================================
# #    N     N   OOO   RRRRRR  M     M    A    L       IIIIIII ZZZZZZZ EEEEEEE     RRRRRR    SSSSS GGGGGGG L       U     U   OOO   N     N     
# #    NN    N  O   O  R     R MM   MM   A A   L          I         Z  E           R     R  S      G       L       U     U  O   O  NN    N     
# #    N N   N O     O R     R M M M M  A   A  L          I        Z   E           R     R S       G       L       U     U O     O N N   N     
# #    N  N  N O     O RRRRRR  M  M  M  AAAAA  L          I       Z    EEEEEEE     RRRRRR   SSSSS  G  GGGG L       U     U O     O N  N  N     
# #    N   N N O     O R   R   M     M A     A L          I      Z     E           R   R         S G     G L       U     U O     O N   N N     
# #    N    NN  O   O  R    R  M     M A     A L          I     Z      E           R    R       S  G     G L        U   U   O   O  N    NN     
# #    N     N   OOO   R     R M     M A     A LLLLLLL IIIIIII ZZZZZZZ EEEEEEE     R     R SSSSS    GGGGG  LLLLLLL   UUU     OOO   N     N 
# #    ===================================================================================================================================

#     RSG1000_hist = RSGluon1000_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'UL16postVFP_RSGluon1000')
#     RSG1000_hist.scale(RSGluon1000_sf2016) #scaled according to luminosity  

# #    ===========================================================================================
# #    N     N   OOO   RRRRRR  M     M    A    L       IIIIIII ZZZZZZZ EEEEEEE     DDDD    M     M     
# #    NN    N  O   O  R     R MM   MM   A A   L          I         Z  E           D   D   MM   MM     
# #    N N   N O     O R     R M M M M  A   A  L          I        Z   E           D    D  M M M M     
# #    N  N  N O     O RRRRRR  M  M  M  AAAAA  L          I       Z    EEEEEEE     D     D M  M  M     
# #    N   N N O     O R   R   M     M A     A L          I      Z     E           D    D  M     M     
# #    N    NN  O   O  R    R  M     M A     A L          I     Z      E           D   D   M     M     
# #    N     N   OOO   R     R M     M A     A LLLLLLL IIIIIII ZZZZZZZ EEEEEEE     DDDD    M     M
# #    ===========================================================================================

#     DM1000_hist = DM1000_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'UL16postVFP_DM1000')
#     DM1000_hist.scale(DM1000_sf2016) #scaled according to luminosity

# #    ===========================================================================
# #    M     M    A    K     K EEEEEEE     PPPPPP  L         OOO   TTTTTTT   SSSSS     
# #    MM   MM   A A   K   K   E           P     P L        O   O     T     S          
# #    M M M M  A   A  K K     E           P     P L       O     O    T    S           
# #    M  M  M  AAAAA  KKk     EEEEEEE     PPPPPP  L       O     O    T     SSSSS      
# #    M     M A     A K  K    E           P       L       O     O    T          S     
# #    M     M A     A K   K   E           P       L        O   O     T         S      
# #    M     M A     A K   K   EEEEEEE     P       LLLLLLL   OOO      T    SSSSS 
# #    ===========================================================================

#     MC_hist = TTbar_hist.copy()
#     MC_hist.add(QCD_hist)
#     # ---- Extract both the data and MC events and from histograms ---- #
#     if ihist == 'SDmass':
#         NtotalMC = np.sum(MC_hist.integrate('jetmass').values())
#         NtotalData = np.sum(Data_hist.integrate('jetmass').values())
#     else:
#         NtotalMC = np.sum(MC_hist.integrate(ihist).values())
#         NtotalData = np.sum(Data_hist.integrate(ihist).values())
#     NtotalMC = [k for k in NtotalMC.values()]
#     NtotalData = [l for l in NtotalData.values()]

#     # ---- Normalize the total MC histogram directly to the data (for aesthetic purposes only!) ---- #
#     # -------- Unweighted simulation of the background alone greatly underestimates the data ------- #
#     if NtotalMC[0] > 0.:
#         MC_hist.scale(NtotalData[0]/NtotalMC[0])
#     else:
#         MC_hist.scale(0.)


#     #---- Plot Data ----#
#     #-----------------------------------------------------------------#
#     hist.plot1d(Data_hist, ax=ax, clear=False, 
#                 error_opts=data_err_opts)

#     #---- Plot Total MC (simulated QCD + SM ttbar background)----#
#     #-----------------------------------------------------------------#
#     hist.plot1d(MC_hist, ax=ax, clear=False,
#                 fill_opts=stack_background_opts, error_opts=stack_error_opts)

#     #---- Plot TTbar MC for comparison ---- #
#     #-----------------------------------------------------------------#    
#     hist.plot1d(TTbar_hist, ax=ax, clear=False,
#                 fill_opts=stack_ttbar_opts, error_opts=stack_error_opts)

#     #---- Plot RSG MC for comparison ---- #
#     #-----------------------------------------------------------------#   
#     hist.plot1d(RSG1000_hist, ax=ax, clear=False,
#                 line_opts=line_rsg1000_opts, error_opts=stack_error_opts, legend_opts=legend_labels)


#     #---- Plot DMM MC for comparison ----#
#     #-----------------------------------------------------------------#
#     hist.plot1d(DM1000_hist, ax=ax, clear=False,
#                 line_opts=line_dm1000_opts, error_opts=stack_error_opts, legend_opts=legend_labels)

#     ax.set_ylim(bottom=0.1)
#     ax.set_yscale('log')
#     ax.set_ylabel('Events')
#     ax.set_xlabel(None)
#     ax.set_title(title)
#     leg = ax.legend(labels=[r'QCD Sim. Bkg', r'$t\bar{t}$ Sim.', r'RSKK Gluon $1TeV$ Sim.', r'DM Med. $1TeV$ Sim.', r'Data'])


#     #---- Plot Ratio ----#
#     hist.plotratio(num = Data_hist, denom = MC_hist, ax = rax,
#                    error_opts={'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1},
#                    unc = 'num')
#     rax.set_ylabel('Data/MC')
#     rax.axhline(y=1, color='k', linestyle=':')
#     rax.set_ylim(0,2)

#     if ihist == 'ttbarmass':
#         rax.set_xlim(700,5000)
#     if ihist == 'jetpt':
#         rax.set_xlim(400,3000)
#     if ihist == 'jeteta':
#         rax.set_xlim(-3,3)
#     if ihist == 'tau32':
#         rax.set_xlim(0,1.2)

#     #---- Labeling ----#
#     Lint = str(Lum2016*.001) # Integrated Luminosity
#     lumi = plt.text(1.15, 1.07, Lint[:6] + " fb$^{-1}$",
#             fontsize=16,
#             horizontalalignment='right',
#             verticalalignment='top',
#             transform=ax.transAxes
#            )
# #         plt.savefig(SaveDirectory+filename, bbox_inches="tight")
# #         print(SaveDirectory+filename)